<a href="https://colab.research.google.com/github/delphi12/VirtualAssistant_Project/blob/main/SuperGlue_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
pip install datasets transformers > /dev/null

In [34]:
squad_v2 = False
model_checkpoint = "deepset/roberta-base-squad2"
batch_size = 32

In [35]:
from datasets import load_dataset, load_metric

In [36]:
datasets = load_dataset("super_glue", "multirc")

  0%|          | 0/3 [00:00<?, ?it/s]

In [37]:
datasets["train"]

Dataset({
    features: ['paragraph', 'question', 'answer', 'idx', 'label'],
    num_rows: 27243
})

In [38]:
datasets["validation"] = datasets["train"]
datasets["validation"] = datasets["validation"].select([i for i in range(0,300)])
datasets["train"] = datasets["train"].select([i for i in range(300,2019)])

In [39]:
datasets["train"][0]

{'paragraph': 'The side of Malaquez\'s parcel gave way to reveal a greenmunk caught in a sheen of solid air. Bits of leaf mold flew from under his feet as he ran to greet a friend or a bringer of food. Tasha oohed in awe. I said, "Frodo\'s been visiting you, eh?" Malaquez said, "Your pet?" "Hardly. He lives around here somewhere. I suppose he was attracted to the commotion up the hill." "Ah," Malaquez said. "Why \'Frodo\'?" Tasha said, "A little fellow with big, furry feet. What else could he be called?" She handed the sculpture to me. I almost dropped it; I expected it to weigh no more than a holo. "Heavy," I said, as if he might not have known. He laughed. "My last piece was of four old Undersiders crouched around a trash fire. Be glad someone didn\'t toss that to you." He spoke of his art with the enthusiasm of a seven-year-old. "Um, I should wait to importune you, but..." ',
 'question': "Is Frodo the narrator's pet?",
 'answer': 'Frodo is a pet',
 'idx': {'paragraph': 5, 'question

In [40]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [41]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [42]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [43]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [0, 2264, 16, 110, 766, 116, 2, 2, 2387, 766, 16, 28856, 1851, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [44]:
max_length = 384
doc_stride = 128

In [45]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["paragraph"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [46]:
len(tokenizer(example["question"], example["paragraph"])["input_ids"])

433

In [47]:
len(tokenizer(example["question"], example["paragraph"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [48]:
tokenized_example = tokenizer(
    example["question"],
    example["paragraph"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [49]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 193]

In [50]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

<s>List some of the effects of the 2010 Deepwater Horizon explosion</s></s>(CNN) -- Declaring 2010 "The best year in safety performance in our company's history," Transocean Ltd., owner of the Gulf of Mexico oil rig that exploded, killing 11 workers, has awarded its top executives hefty bonuses and raises, according to a recent filing with the U.S. Securities and Exchange Commission. That includes a $200,000 salary increase for Transocean president and chief executive officer Steven L. Newman, whose base salary will increase from $900,000 to $1.1 million, according to the SEC report. Newman's bonus was $374,062, the report states. Newman also has a $5.4 million long-term compensation package the company awarded him upon his appointment as CEO in March 2010, according to the SEC filing. The latest cash awards are based in part on the company's "performance under safety," the Transocean filing states. "Notwithstanding the tragic loss of life in the Gulf of Mexico, we achieved an exemplar

In [51]:
tokenized_example = tokenizer(
    example["question"],
    example["paragraph"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 9), (10, 12), (13, 16), (17, 24), (25, 27), (28, 31), (32, 36), (37, 41), (41, 46), (47, 54), (55, 64), (0, 0), (0, 0), (0, 1), (1, 4), (4, 5), (6, 8), (9, 13), (13, 18), (19, 23), (24, 25), (25, 28), (29, 33), (34, 38), (39, 41), (42, 48), (49, 60), (61, 63), (64, 67), (68, 75), (75, 77), (78, 85), (85, 87), (88, 93), (93, 94), (94, 98), (99, 102), (102, 104), (105, 110), (111, 113), (114, 117), (118, 122), (123, 125), (126, 132), (133, 136), (137, 140), (141, 145), (146, 154), (154, 155), (156, 163), (164, 166), (167, 174), (174, 175), (176, 179), (180, 187), (188, 191), (192, 195), (196, 206), (207, 212), (213, 220), (221, 224), (225, 231), (231, 232), (233, 242), (243, 245), (246, 247), (248, 254), (255, 261), (262, 266), (267, 270), (271, 272), (272, 273), (273, 274), (274, 275), (276, 286), (287, 290), (291, 299), (300, 310), (310, 311), (312, 316), (317, 325), (326, 327), (328, 329), (329, 332), (332, 333), (333, 336), (337, 343), (344, 352), (353, 356), (35

In [52]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

List List


In [53]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [54]:
answers = example["answer"]
start_char = example["idx"]["answer"]
end_char = start_char + len(answers)

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

107 109


In [55]:
pad_on_right = tokenizer.padding_side == "right"

In [56]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "paragraph"],
        examples["paragraph" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]
        #example["idx"]["answer"]
        # If no answers are given, set the cls_index as answer.
        if (example["idx"]["answer"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = example["idx"]["answer"]
            end_char = start_char + len(answers)

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [57]:
features = prepare_train_features(datasets['train'][:10])

In [58]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/1719 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/9693 [00:00<?, ? examples/s]

In [60]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [61]:
args = TrainingArguments(
    f"test-superglue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

In [62]:
from transformers import default_data_collator

data_collator = default_data_collator

In [63]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [64]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [65]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([32, 384]), torch.Size([32, 384]))

In [66]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0, 195,   0, 195,   0, 195,   0, 195,   0, 195,   0,
         195,   0, 246,  30], device='cuda:0'),
 tensor([ 90,   0,  90,   0,  90,   0,  90,   0,  90,   0,  90,   0,  90,   0,
          90,   0,  90,   0, 197,   0, 197,   0, 197,   0, 197,   0, 197,   0,
         197,   0, 278,  52], device='cuda:0'))

In [67]:
n_best_size = 20

In [68]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [69]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "paragraph"],
        examples["paragraph" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["idx"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [70]:
validation_features = datasets["validation"].map(prepare_validation_features, batched=True, remove_columns=datasets["validation"].column_names)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [71]:
raw_predictions = trainer.predict(validation_features)

In [72]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [73]:
max_answer_length = 30

In [74]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["paragraph"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 2.0707114, 'text': 'use its influence over the Taliban'},
 {'score': 1.2083018, 'text': 'influence over the Taliban'},
 {'score': 0.5467043, 'text': 'Taliban'},
 {'score': 0.3767476, 'text': 'to use its influence over the Taliban'},
 {'score': 0.08254659, 'text': 'the Taliban'},
 {'score': -0.0009371042, 'text': 'its influence over the Taliban'},
 {'score': -0.54791385, 'text': 'Bin Laden'},
 {'score': -1.1870579,
  'text': 'United States began a high-level effort to persuade Pakistan to use its influence over the Taliban'},
 {'score': -1.4459144,
  'text': 'the United States began a high-level effort to persuade Pakistan to use its influence over the Taliban'},
 {'score': -2.2549922, 'text': 'use its influence'},
 {'score': -2.944878, 'text': ' Bin Laden'},
 {'score': -2.9859562, 'text': 'use its influence over the Taliban.'},
 {'score': -3.094065, 'text': 'Direct pressure on the Taliban'},
 {'score': -3.1174018, 'text': 'influence'},
 {'score': -3.1791785, 'text': 'Taliban

In [75]:
import collections
from collections import defaultdict
examples = datasets["validation"]
features = validation_features
example_id_to_index ={}

for i in range(0,301):
  temp=features["example_id"][i]["answer"]
  example_id_to_index[i] = temp
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
  temp=features["example_id"][i]["answer"]
  features_per_example[example_id_to_index[temp]].append(i)

In [76]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
   
    example_id_to_index ={}

    for i in range(0,301):
      temp=features["example_id"][i]["answer"]
      example_id_to_index[i] = temp
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
      temp=features["example_id"][i]["answer"]
    features_per_example[example_id_to_index[temp]].append(i)

   
    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    
    for example_index, example in enumerate(tqdm(examples)):

        feature_indices = features_per_example[example_index]

        min_null_score = None 
        valid_answers = []
        
        context = example["paragraph"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        if not squad_v2:
            predictions[example["answer"]] = best_answer["text"]
        else:
          answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
          predictions[example["id"]] = answer

    return predictions

In [77]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 300 example predictions split into 468 features.


  0%|          | 0/300 [00:00<?, ?it/s]

In [80]:
mylist =[]
for x in range(0, 301):
    mylist.append(x)

In [81]:
for x in range(259, 301):
  final_predictions.update({x : "Null values"+str(x)})

In [82]:
my_predictions = list(final_predictions.keys())
print(my_predictions)

['Children, Gerd, or Dorian Popa', 'Monetary rewards', 'Asking Pakistan to help the USA', 'Meeting with General Musharraf', 'President Clinton offered the moon in terms of better relations with the United States', 'A Presidential visit in March', 'Paper checks', 'Increasing trade with Pakistan', 'Persuading Pakistan to use its influence over the Taliban by dangling before him the possibility of a presidential visit in March as a reward for Pakistani cooperation', 'Romanian police or secret service', 'President Cliton', 'Karl Inderfurth', 'Michael Sheehan', 'The department of counterterrorism', 'Mullah Omar', 'No', 'Yes', 'Yes there was no disagreement between the two', "No, the CIA felt it would risk the president's life and they felt that Pakistan had not done enough to merit a presidential visit", '11.09.2020', 'March 25, 1999', 'March 15, 2000', '12.03.2019', 'January 2000', 'March 25, 2000', 'Direct pressure on Taliban had proved unsuccessful', 'The envoys reported that Pakistan wa

In [83]:
def f1_score(prediction, ground_truth):
    prediction_tokens = prediction.split()
    ground_truth_tokens = ground_truth.split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (prediction == ground_truth)

In [84]:
orig = list(datasets["validation"])
for i in range(0,len(orig)):
  orig[i] = orig[i]["answer"]

In [85]:
em = 0
for i in range(0,len(my_predictions)):
  if orig[i] == my_predictions[i]:
    em+=1 
print("Exact Match {}".format(em/len(orig)))

Exact Match 0.12333333333333334


In [86]:
from sklearn.metrics import f1_score
f1 = f1_score(orig, my_predictions, average='macro')
print("f1_score {}".format(f1))

f1_score 0.10522222222222222
